# 検索速度の比較
Valdを使った場合とアプリケーション側でベクトル距離計算をする場合とでデータ量を変え速度を比較してみます。

## データの準備

ユニーク処理後に1M程度のデータを日本語では見つけられなかったため、以下の英語wikipedia文章のデータセットを使います。メモリを大量に使い時間もかかるので、まずはスライス等で小さいデータにしてから試すことをお勧めします。

https://huggingface.co/datasets/wikitext

In [ ]:
from datasets import load_dataset

dataset = load_dataset("wikitext", "wikitext-103-raw-v1", split="train")

In [31]:
df = pd.DataFrame(data=dataset["text"], columns=["text"])

In [32]:
len(df)

1801350

In [34]:
# 重複を除去
df = df.drop_duplicates(subset="text", keep="first", ignore_index=True)

In [38]:
len(df)

973414

In [36]:
df.to_csv("./wikitext-uniq.csv", index=False, quoting=1)

## 文章のベクトル化
高速化のため、ベクトル化にはOpenAI Embeddings APIではなくsentence-transformersを使います。

In [6]:
import pandas as pd

df = pd.read_csv("./wikitext-uniq.csv")

In [7]:
from sentence_transformers import SentenceTransformer

# CPUを使用する場合
# model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

# GPUを使用する場合
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2', device='cuda')

In [8]:
def get_embedding(text, model):
    return model.encode(text)

In [ ]:
from tqdm.notebook import tqdm

tqdm.pandas()

df["text_embedding"] = df["text"].progress_apply(lambda x: get_embedding(x, model))

In [ ]:
# fileから読み込んだあともembeddingが復元できるよう加工して保存しています
w_df = df.copy()
w_df["title_embedding"] = w_df["title_embedding"].apply(list)
w_df.to_csv("./wikitext-uniq-with-text-embedding.csv", index=False)

## クエリ速度の比較

In [2]:
import pandas as pd

df = pd.read_csv("./wikitext-uniq-with-text-embedding.csv")

In [ ]:
import numpy as np
from tqdm.notebook import tqdm

tqdm.pandas()

df["text_embedding"] = (
    df["text_embedding"].progress_apply(eval).progress_apply(np.array)
)

### ベクトル検索エンジンを使わない場合

In [9]:
def get_insert_features(df):
    insert_features = np.array([x for x in df["text_embedding"].values])
    return insert_features

In [10]:
def get_indexes_top_k_with_numpy(insert_features, query_feature, k):
    distances = np.linalg.norm(
        query_feature - insert_features, axis=1
    )  # distance_type=L2と等価
    distance_indexes = np.argsort(distances)[:k]

    return distance_indexes

In [11]:
text = "野原ひろしの住んでいる地域はどこですか"
query_feature = get_embedding(text=text, model=model)

#### データ数1万

In [12]:
insert_features = get_insert_features(df[:10000])

In [13]:
%%time
indexes = get_indexes_top_k_with_numpy(insert_features, query_feature, k=3)

CPU times: user 21 ms, sys: 35 µs, total: 21 ms
Wall time: 19.8 ms


#### データ数10万

In [14]:
insert_features = get_insert_features(df[:100000])

In [15]:
%%time
indexes = get_indexes_top_k_with_numpy(insert_features, query_feature, k=3)

CPU times: user 182 ms, sys: 952 µs, total: 183 ms
Wall time: 181 ms


#### データ数97万

In [16]:
insert_features = get_insert_features(df)

In [17]:
%%time
indexes = get_indexes_top_k_with_numpy(insert_features, query_feature, k=3)

CPU times: user 1.38 s, sys: 973 ms, total: 2.36 s
Wall time: 2.36 s


### ベクトル検索エンジンValdを使った場合
この環境では外部通信を行なっているためネットワークによる遅延が入っています。1番最初の通信はコネクション確立に時間がかかるため、実際使われる時の状況に近い2回目の速度を測っています。

In [14]:
import grpc
from vald.v1.payload import payload_pb2
from vald.v1.vald import search_pb2_grpc, upsert_pb2_grpc

In [15]:
## 接続先Host名(Host:Port)
host = "your-vald-host:80"

## 次元数
dimension = 768

In [16]:
channel = grpc.insecure_channel(host)

In [17]:
usstub = upsert_pb2_grpc.UpsertStub(channel)
sstub = search_pb2_grpc.SearchStub(channel)

In [18]:
from tqdm.notebook import tqdm

uscfg = payload_pb2.Upsert.Config(skip_strict_exist_check=True)


def multi_upsert(df, chunk_size=200):
    for i in tqdm(range(0, len(df), chunk_size)):
        requests = [
            payload_pb2.Upsert.Request(
                vector=payload_pb2.Object.Vector(
                    id=str(row.Index), vector=row.text_embedding
                ),
                config=uscfg,
            )
            for row in df[i : i + chunk_size].itertuples()
        ]
        usstub.MultiUpsert(payload_pb2.Upsert.MultiRequest(requests=requests))

In [19]:
def get_indexes_top_k(vec, k):
    scfg = payload_pb2.Search.Config(
        num=k, radius=-1.0, epsilon=0.01, timeout=3000000000
    )
    response = sstub.Search(payload_pb2.Search.Request(vector=vec, config=scfg))
    return [int(result.id) for result in response.results]

In [ ]:
# 最初の通信用
text = "テスト用のクエリです"
query_feature = get_embedding(text=text, model=model)
multi_upsert(df[:10])

In [32]:
indexes = get_indexes_top_k(query_feature, k=3)

#### データ数1万

In [ ]:
multi_upsert(df[:10000])

In [33]:
text = "野原ひろしの住んでいる地域はどこですか"
query_feature = get_embedding(text=text, model=model)

In [42]:
%%time
indexes = get_indexes_top_k(query_feature, k=3)

CPU times: user 0 ns, sys: 2.81 ms, total: 2.81 ms
Wall time: 108 ms


#### データ数10万

In [ ]:
multi_upsert(df[10000:100000])

In [59]:
%%time
indexes = get_indexes_top_k(query_feature, k=3)

CPU times: user 2.27 ms, sys: 10 µs, total: 2.28 ms
Wall time: 105 ms


#### データ数97万

In [ ]:
multi_upsert(df[100000:])

In [34]:
%%time
indexes = get_indexes_top_k(query_feature, k=3)

CPU times: user 2.94 ms, sys: 43 µs, total: 2.98 ms
Wall time: 104 ms


検索速度はベクトルの分布や設定にも依存しますが、社内で運用している環境ではデータ数が1000万件以上でもsearchの99%ile値は200ms以下となっています。

## 謝辞
データセットに用いたwikitextは以下ライセンスに基づき改変せず使用させていただきました。

https://creativecommons.org/licenses/by-sa/4.0/deed.ja

データを公開いただきましたWikipedia様とデータセットを作成いただいたStephen Merity、Caiming Xiong、James Bradbury、Richard Socher様に感謝を申し上げます。